### Импорт зависимостей

In [34]:
import os
import random
import shutil
from pathlib import Path

import kagglehub
from kagglehub import KaggleDatasetAdapter

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory


### Загрузка датасета

In [17]:
dataset_path = kagglehub.dataset_download('harishekar/chess-board-state-detection-dataset')
dataset_path

'C:\\Users\\rayshite\\.cache\\kagglehub\\datasets\\harishekar\\chess-board-state-detection-dataset\\versions\\1'

### Разделение данных на обучающюю, валидационную и тестовую выборки

In [19]:
SRC_DIR = Path(dataset_path) / "final_original_dataset"
DEST_DIR = Path("data")

# доли выборок
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

random.seed(42)

for class_dir in SRC_DIR.iterdir():
    if not class_dir.is_dir():
        continue

    class_name = class_dir.name 
    images = [p for p in class_dir.iterdir() if p.is_file()]
    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    n_test = n_total - n_train - n_val  # остаток уходит в test

    splits = {
        "train": images[:n_train],
        "val":   images[n_train:n_train + n_val],
        "test":  images[n_train + n_val:],
    }

    for split_name, split_files in splits.items():
        target_dir = DEST_DIR / split_name / class_name
        target_dir.mkdir(parents=True, exist_ok=True)

        for src_path in split_files:
            dst_path = target_dir / src_path.name
            shutil.copy2(src_path, dst_path)

    print(f"{class_name}: total={n_total}, train={n_train}, val={n_val}, test={n_test}")

bB: total=100, train=70, val=15, test=15
bK: total=100, train=70, val=15, test=15
bN: total=100, train=70, val=15, test=15
bP: total=100, train=70, val=15, test=15
bQ: total=100, train=70, val=15, test=15
bR: total=100, train=70, val=15, test=15
empty: total=100, train=70, val=15, test=15
wB: total=100, train=70, val=15, test=15
wK: total=100, train=70, val=15, test=15
wN: total=100, train=70, val=15, test=15
wP: total=100, train=70, val=15, test=15
wQ: total=100, train=70, val=15, test=15
wR: total=100, train=70, val=15, test=15


### Создание базовой модели

In [48]:
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(13, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [50]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 13)             │       163,085 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,141,581 (4.35 MB)

 Trainable params: 1,141,581 (4.35 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model.compile(loss="sparse_categorical_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"])